In [658]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer

In [659]:
titanicData = pd.read_csv('http://bit.ly/kaggletrain', index_col = None)
titanicData

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [660]:
titanicData.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [661]:
titanicData.shape

(891, 12)

In [662]:
titanicData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [663]:
cols = ["Pclass", "Name", "Sex", "Age", "SibSp", "Parch", "Embarked", "Fare"]
titanicData_X = titanicData[cols]
titanicData_y = titanicData["Survived"]
titanicData_X

,Pclass,Name,Sex,Age,SibSp,Parch,Embarked,Fare
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,S,7.2500
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,C,71.2833
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,S,7.9250
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,S,53.1000
4,3,"Allen, Mr. William Henry",male,35.0,0,0,S,8.0500
...,...,...,...,...,...,...,...,...
886,2,"Montvila, Rev. Juozas",male,27.0,0,0,S,13.0000
887,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,S,30.0000
888,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,S,23.4500
889,1,"Behr, Mr. Karl Howell",male,26.0,0,0,C,30.0000


In [664]:
titanicData_y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [665]:
titanicData_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Name      891 non-null    object 
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Embarked  889 non-null    object 
 7   Fare      891 non-null    float64
dtypes: float64(2), int64(3), object(3)
memory usage: 55.8+ KB


### Required transformations
- Pclass: None
- Name: CountVectorizer
- Sex: Categorical to Numerical
- Age: Missing value, To numerical
- Embarked: To numerical
- Fare: None

### Steps
1- "Pclass", "Name", "Sex", "SibSp", "Parch", "Fare"
2- "Pclass", "Name", "Sex", "SibSp", "Parch", "Fare", "Age"
3- "Pclass", "Name", "Sex", "SibSp", "Parch", "Fare", "Age", "Embarked"

In [666]:
step1Cols = ["Pclass", "Name", "Sex", "SibSp", "Parch", "Fare"]
step2Cols = ["Pclass", "Name", "Sex", "SibSp", "Parch", "Fare","Age"]
step3Cols = ["Pclass", "Name", "Sex", "SibSp", "Parch", "Fare", "Age", "Embarked"]

### Step 1

In [667]:
knn = KNeighborsClassifier(n_neighbors = 8)
X_train, X_test, y_train, y_test = train_test_split(
    titanicData_X[step1Cols], titanicData_y,
    random_state = 40, test_size = 0.2
)

In [668]:
X_train.shape

(712, 6)

In [669]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 661 to 326
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  712 non-null    int64  
 1   Name    712 non-null    object 
 2   Sex     712 non-null    object 
 3   SibSp   712 non-null    int64  
 4   Parch   712 non-null    int64  
 5   Fare    712 non-null    float64
dtypes: float64(1), int64(3), object(2)
memory usage: 38.9+ KB


In [670]:
y_train.shape

(712,)

###

In [671]:
countVectorizer = CountVectorizer()
oneHotEncoder = OneHotEncoder()

In [672]:
step1_columnTransformer = make_column_transformer(
    (countVectorizer, "Name"),
    (oneHotEncoder, ["Sex"]),
    remainder = "passthrough",
    n_jobs = -1
)

In [673]:
step1_pipeLine = make_pipeline(step1_columnTransformer, knn)

In [674]:
k_range = list(range(1, 31))
params = {"kneighborsclassifier__n_neighbors": k_range}

In [675]:
gridSearchCV = GridSearchCV(
    step1_pipeLine, params, cv = 5, scoring = "accuracy", n_jobs = -1
)

In [676]:
gridSearchCV.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(n_jobs=-1,
                                                          remainder='passthrough',
                                                          transformers=[('countvectorizer',
                                                                         CountVectorizer(),
                                                                         'Name'),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(),
                                                                         ['Sex'])])),
                                       ('kneighborsclassifier',
                                        KNeighborsClassifier(n_neighbors=8))]),
             n_jobs=-1,
             param_grid={'kneighborsclassifier__n_neighbors': [1, 2,

In [677]:
gridSearchCV.best_score_

0.7626514330739682

In [678]:
predicted = gridSearchCV.predict(X_test)

In [679]:
print("Step 1 Accuracy:", accuracy_score(y_test, predicted))

Step 1 Accuracy: 0.8324022346368715


### End - Step 1

### Step 2

In [680]:
knn = KNeighborsClassifier(n_neighbors = 8)
X_train, X_test, y_train, y_test = train_test_split(
    titanicData_X[step2Cols], titanicData_y,
    random_state = 40, test_size = 0.2
)

In [681]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 661 to 326
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Pclass  712 non-null    int64  
 1   Name    712 non-null    object 
 2   Sex     712 non-null    object 
 3   SibSp   712 non-null    int64  
 4   Parch   712 non-null    int64  
 5   Fare    712 non-null    float64
 6   Age     573 non-null    float64
dtypes: float64(2), int64(3), object(2)
memory usage: 44.5+ KB


In [682]:
X_train.isna().sum()

Pclass      0
Name        0
Sex         0
SibSp       0
Parch       0
Fare        0
Age       139
dtype: int64

In [683]:
step2_simpleImputer = SimpleImputer()

In [684]:
step2_columnTransformer = make_column_transformer(
    (countVectorizer, "Name"),
    (oneHotEncoder, ["Sex"]),
    (step2_simpleImputer, ["Age"]),
    remainder = "passthrough",
    n_jobs = -1
)

In [685]:
step2_pipeLine = make_pipeline(
    step2_columnTransformer, knn
)

In [686]:
step2_gridSearchCV = GridSearchCV(
    step2_pipeLine, params, cv = 5, scoring = "accuracy", n_jobs = -1
)

In [687]:
step2_gridSearchCV.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(n_jobs=-1,
                                                          remainder='passthrough',
                                                          transformers=[('countvectorizer',
                                                                         CountVectorizer(),
                                                                         'Name'),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(),
                                                                         ['Sex']),
                                                                        ('simpleimputer',
                                                                         SimpleImputer(),
                                                           

In [688]:
step2_gridSearchCV.best_score_

0.719078104993598

In [689]:
predicted = step2_gridSearchCV.predict(X_test)
print("Step 2 Accuracy:", accuracy_score(y_test, predicted))

Step 2 Accuracy: 0.7318435754189944


### End - Step2

In [690]:
knn = KNeighborsClassifier(n_neighbors = 8)
X_train, X_test, y_train, y_test = train_test_split(
    titanicData_X[step3Cols], titanicData_y,
    random_state = 40, test_size = 0.2
)

In [691]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 661 to 326
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    712 non-null    int64  
 1   Name      712 non-null    object 
 2   Sex       712 non-null    object 
 3   SibSp     712 non-null    int64  
 4   Parch     712 non-null    int64  
 5   Fare      712 non-null    float64
 6   Age       573 non-null    float64
 7   Embarked  711 non-null    object 
dtypes: float64(2), int64(3), object(3)
memory usage: 66.2+ KB


In [693]:
step3ConstantImputer = SimpleImputer(
    strategy = "constatnt",
    fill_value = "missing"
)

In [697]:
ageTransformer = make_pipeline(step3ConstantImputer, oneHotEncoder)

In [698]:
step3_columnTransformer = make_column_transformer(
    (countVectorizer, "Name"),
    (oneHotEncoder, ["Sex"]),
    (step2_simpleImputer, ["Age"]),
    (ageTransformer, ["Embarked"]),
    remainder = "passthrough",
    n_jobs = -1
)